<a href="https://colab.research.google.com/github/gcosma/DECODEclinicalTrialCalc/blob/main/HospitalisationCalculator20Feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hospitalisation clinical calculator - version 20 Feb 2025

Summary of overall admissions Admission rates (per 1000 persons per year) by type are shown in Figure 29. The overall rate for adults with ID was 351.6 per 1000 persons per year, compared with 246.4 per 1000 persons per year for controls. This difference was essentially due to the higher rate among emergency admissions (182.2 vs. 67.7 per 1000 persons per year), as elective rates were similar between groups." p70, Carey et al (2017)"


Two-sample Poisson Ratio Tests (Equal Sizes)

In [ ]:
# @title Calculator
import numpy as np
from scipy import stats
from IPython.display import display, HTML
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt

def calculate_achieved_power(N, lambda1, lambda2, sig_level=0.05, alternative='two-sided'):
    """Calculate achieved power for given sample size and parameters"""
    rate_ratio = lambda1 / lambda2

    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level / 2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    ncp = np.log(rate_ratio) / np.sqrt(1 / (lambda1 * N) + 1 / (lambda2 * N))

    if alternative == 'two-sided':
        power = (1 - stats.norm.cdf(z_alpha - ncp) +
                 stats.norm.cdf(-z_alpha - ncp))
    else:
        power = 1 - stats.norm.cdf(z_alpha - ncp)

    return power

def get_clinical_interpretation(lambda1, lambda2, achieved_power, target_power, example_N, attrition_rate):
    """Provide comprehensive clinical interpretation of the results"""
    rate_ratio = lambda1 / lambda2
    events_per_100py_1 = lambda1 * 100
    events_per_100py_2 = lambda2 * 100
    absolute_diff = (lambda1 - lambda2) * 100
    effective_N = example_N * (1 - attrition_rate)

    interpretation = f"""
    <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 5px solid #34495e;'>
        <h3 style='color: #34495e; margin-top: 0;'>Clinical Trial Status Summary</h3>

        <h4 style='color: #2c3e50;'>Recruitment Needs</h4>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
            <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                <h5 style='color: #2c3e50; margin-top: 0;'>Current Status</h5>
                <p>
                • Currently enrolled: {example_N} per arm<br>
                • After {attrition_rate:.0%} dropout: {effective_N} per arm<br>
                • Total enrolled: {example_N * 2} participants
                </p>
            </div>
            <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
                <h5 style='color: #2c3e50; margin-top: 0;'>Power Assessment</h5>
                <p>
                • Target power: {target_power:.1%}<br>
                • Current power: {achieved_power:.1%}<br>
                • Status: {achieved_power >= target_power and "Sufficient" or "Insufficient"}
                </p>
            </div>
        </div>

        <h4 style='color: #2c3e50;'>Expected Event Rates:</h4>
        <p>
        • <strong>Treatment Group:</strong> {events_per_100py_1:.1f} events per 100 person-years<br>
        • <strong>Control Group:</strong> {events_per_100py_2:.1f} events per 100 person-years<br>
        • <strong>Absolute Difference:</strong> {abs(absolute_diff):.1f} events per 100 person-years
        ({absolute_diff:+.1f} change)
        </p>

        <h4 style='color: #2c3e50;'>Projected Clinical Impact:</h4>
        <p>
        For every 100 patients followed for one year:<br>
        • <strong>Treatment Group:</strong> {events_per_100py_1:.0f} expected events<br>
        • <strong>Control Group:</strong> {events_per_100py_2:.0f} expected events<br>
        • <strong>Potential Impact:</strong> Prevention of {abs(absolute_diff):.0f} events per 100 patient-years
        </p>

        <h4 style='color: #2c3e50;'>Power Analysis Status:</h4>"""

    if achieved_power >= target_power:
        interpretation += f"""
        <p style='background-color: #e8f5e9; padding: 10px; border-radius: 5px;'>
        <strong style='color: #27ae60;'>✓ SUFFICIENT POWER</strong><br>
        • Current enrollment provides {achieved_power:.1%} power<br>
        • Exceeds target power of {target_power:.1%}<br>
        • No additional recruitment needed
        </p>"""
    else:
        interpretation += f"""
        <p style='background-color: #ffebee; padding: 10px; border-radius: 5px;'>
        <strong style='color: #c0392b;'>⚠ INSUFFICIENT POWER</strong><br>
        • Current power ({achieved_power:.1%}) below target ({target_power:.1%})<br>
        • Additional recruitment may be needed to reach target power
        </p>"""

    return interpretation

def plot_power_vs_sample_size(lambda1, lambda2, target_power, sig_level, attrition_rate):
    """Plot power vs. sample size for a range of sample sizes"""
    sample_sizes = np.arange(50, 1001, 10)  # Sample sizes from 50 to 1000
    powers = [calculate_achieved_power(N * (1 - attrition_rate), lambda1, lambda2, sig_level)
              for N in sample_sizes]

    plt.figure(figsize=(10, 6))
    plt.plot(sample_sizes, powers, label='Achieved Power', color='blue', linewidth=2)
    plt.axhline(y=target_power, color='red', linestyle='--', label='Target Power')
    plt.xlabel('Sample Size per Arm', fontsize=12)
    plt.ylabel('Power', fontsize=12)
    plt.title('Power vs. Sample Size', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.show()

def display_clinical_power_analysis(example_N, lambda1, lambda2, target_power=0.90, sig_level=0.05, attrition_rate=0.10):
    """Display comprehensive power analysis results with clear interpretation"""
    # Calculate effective N and achieved power
    effective_N = example_N * (1 - attrition_rate)
    achieved_power = calculate_achieved_power(effective_N, lambda1, lambda2, sig_level)

    # Get clinical interpretation
    interpretation = get_clinical_interpretation(
        lambda1, lambda2, achieved_power, target_power, example_N, attrition_rate
    )

    html_output = f"""
    <div style='background-color: #f5f5f5; padding: 20px; border-radius: 10px; font-family: Arial, sans-serif;'>
        <h2 style='color: #2c3e50; border-bottom: 2px solid #2c3e50; padding-bottom: 10px;'>
            Clinical Trial Power Analysis Dashboard
        </h2>

        {interpretation}

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Technical Study Parameters</h3>
            <p>
            • <strong>Study Design:</strong> Two-arm parallel group comparison<br>
            • <strong>Primary Analysis:</strong> Comparison of event rates between groups<br>
            • <strong>Statistical Test:</strong> Two-sided test at {sig_level:.0%} significance level<br>
            • <strong>Expected Dropout Rate:</strong> {attrition_rate:.0%}<br>
            • <strong>Target Power:</strong> {target_power:.1%}<br>
            • <strong>Current Power:</strong> {achieved_power:.1%}
            </p>
        </div>
    </div>
    """
    display(HTML(html_output))

    # Plot power vs. sample size
    plot_power_vs_sample_size(lambda1, lambda2, target_power, sig_level, attrition_rate)

# Set up the interactive widget with default values
@interact(
    example_N=widgets.IntText(
        value=181,
        description='Current enrollment per arm:',
        style={'description_width': '180px'}
    ),
    attrition_rate=widgets.BoundedFloatText(
        value=0.10,
        min=0.0,
        max=0.50,
        step=0.01,
        description='Expected dropout rate:',
        style={'description_width': '180px'}
    ),
    target_power=widgets.BoundedFloatText(
        value=0.80,
        min=0.70,
        max=0.99,
        step=0.01,
        description='Target power:',
        style={'description_width': '180px'}
    )
)
def run_clinical_analysis(example_N, attrition_rate, target_power):
    """Run the power analysis with the specified parameters"""
    display_clinical_power_analysis(
        example_N=example_N,
        lambda1=0.1822,  # Treatment group event rate
        lambda2=0.0677,  # Control group event rate
        target_power=target_power,
        sig_level=0.05,
        attrition_rate=attrition_rate
    )

interactive(children=(IntText(value=181, description='Current enrollment per arm:', style=DescriptionStyle(des…

In [14]:
# @title Enhanced Power Calculator for Hospitalization Rate Reduction
import numpy as np
from scipy import stats
from IPython.display import display, HTML
from ipywidgets import interact, widgets, Layout
import matplotlib.pyplot as plt

def calculate_achieved_power(N, lambda1, lambda2, sig_level=0.05, alternative='two-sided'):
    """Calculate achieved power for given sample size and parameters"""
    rate_ratio = lambda1 / lambda2

    if alternative == 'two-sided':
        z_alpha = stats.norm.ppf(1 - sig_level / 2)
    else:
        z_alpha = stats.norm.ppf(1 - sig_level)

    ncp = np.log(rate_ratio) / np.sqrt(1 / (lambda1 * N) + 1 / (lambda2 * N))

    if alternative == 'two-sided':
        power = (1 - stats.norm.cdf(z_alpha - ncp) +
                stats.norm.cdf(-z_alpha - ncp))
    else:
        power = 1 - stats.norm.cdf(z_alpha - ncp)

    return power

def display_clinical_power_analysis(example_N, baseline_rate, reduction_percent,
                                  target_power=0.90, sig_level=0.05,
                                  attrition_rate=0.10, attrition_timepoints=1):
    """
    Display comprehensive power analysis results with clear interpretation

    Parameters:
    -----------
    example_N : int
        Sample size per arm
    baseline_rate : float
        Initial hospitalization rate
    reduction_percent : float
        Expected percent reduction in hospitalization (e.g., 20 for 20% reduction)
    target_power : float
        Desired statistical power
    sig_level : float
        Significance level
    attrition_rate : float
        Expected dropout rate per timepoint
    attrition_timepoints : int
        Number of timepoints where attrition occurs
    """
    # Calculate lambda values
    lambda1 = baseline_rate
    lambda2 = baseline_rate * (1 - reduction_percent/100)

    # Calculate cumulative attrition
    cumulative_attrition = 1 - (1 - attrition_rate)**attrition_timepoints
    effective_N = example_N * (1 - cumulative_attrition)

    # Calculate power
    achieved_power = calculate_achieved_power(effective_N, lambda1, lambda2, sig_level)

    # Calculate event rates per 100 person-years
    events_per_100py_1 = lambda1 * 100
    events_per_100py_2 = lambda2 * 100
    absolute_diff = (lambda1 - lambda2) * 100

    html_output = f"""
    <div style='background-color: #f5f5f5; padding: 20px; border-radius: 10px; font-family: Arial, sans-serif;'>
        <h2 style='color: #2c3e50; border-bottom: 2px solid #2c3e50; padding-bottom: 10px;'>
            Hospitalization Rate Reduction Study - Power Analysis
        </h2>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Sample Size Information</h3>
            <p>
            • <strong>Initial Sample Size:</strong> {example_N} per arm (Total: {example_N * 2})<br>
            • <strong>Attrition Rate:</strong> {attrition_rate:.1%} per timepoint<br>
            • <strong>Number of Attrition Timepoints:</strong> {attrition_timepoints}<br>
            • <strong>Cumulative Attrition:</strong> {cumulative_attrition:.1%}<br>
            • <strong>Final Effective Sample Size:</strong> {effective_N:.0f} per arm
            </p>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Statistical Parameters</h3>
            <p>
            • <strong>Target Power:</strong> {target_power:.1%}<br>
            • <strong>Achieved Power:</strong> {achieved_power:.1%}<br>
            • <strong>Significance Level:</strong> {sig_level:.0%}
            </p>
        </div>

        <div style='background-color: #fff; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Expected Event Rates</h3>
            <p>
            • <strong>Baseline Rate:</strong> {events_per_100py_1:.2f} events per 100 person-years<br>
            • <strong>Expected Rate After Treatment:</strong> {events_per_100py_2:.2f} events per 100 person-years<br>
            • <strong>Absolute Reduction:</strong> {abs(absolute_diff):.2f} events per 100 person-years<br>
            • <strong>Relative Reduction:</strong> {reduction_percent:.1f}%
            </p>
        </div>

        <div style='background-color: {achieved_power >= target_power and "#e8f5e9" or "#ffebee"}; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #2c3e50; margin-top: 0;'>Power Analysis Status</h3>
            <p>
            <strong>{achieved_power >= target_power and "✓ SUFFICIENT POWER" or "⚠ INSUFFICIENT POWER"}</strong><br>
            Current power ({achieved_power:.1%}) is {achieved_power >= target_power and "above" or "below"} target power ({target_power:.1%})
            </p>
        </div>
    </div>
    """
    display(HTML(html_output))

    # Plot power vs. sample size
    sample_sizes = np.arange(50, 1001, 10)
    powers = [calculate_achieved_power(N * (1 - cumulative_attrition), lambda1, lambda2, sig_level)
              for N in sample_sizes]

    plt.figure(figsize=(10, 6))
    plt.plot(sample_sizes, powers, label='Achieved Power', color='blue', linewidth=2)
    plt.axhline(y=target_power, color='red', linestyle='--', label='Target Power')
    plt.axvline(x=example_N, color='green', linestyle='--', label='Current Sample Size')
    plt.xlabel('Initial Sample Size per Arm', fontsize=12)
    plt.ylabel('Power', fontsize=12)
    plt.title(f'Power vs. Sample Size for Detecting {reduction_percent}% Reduction\n(Including {cumulative_attrition:.1%} Cumulative Attrition)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.show()

# Set up the interactive widget with improved controls
@interact(
    example_N=widgets.IntText(
        value=181,
        description='Sample size per arm:',
        style={'description_width': '180px'}
    ),
    baseline_rate=widgets.FloatText(
        value=0.1822,
        description='Baseline hosp. rate:',
        style={'description_width': '180px'}
    ),
    reduction_percent=widgets.FloatText(
        value=20.0,
        description='Expected reduction %:',
        style={'description_width': '180px'}
    ),
    attrition_rate=widgets.BoundedFloatText(
        value=0.10,
        min=0.0,
        max=0.50,
        step=0.01,
        description='Attrition rate per timepoint:',
        style={'description_width': '180px'}
    ),
    attrition_timepoints=widgets.IntText(
        value=1,
        min=1,
        description='Number of attrition timepoints:',
        style={'description_width': '180px'}
    ),
    target_power=widgets.BoundedFloatText(
        value=0.80,
        min=0.70,
        max=0.99,
        step=0.01,
        description='Target power:',
        style={'description_width': '180px'}
    )
)
def run_hospitalization_analysis(example_N, baseline_rate, reduction_percent,
                               attrition_rate, attrition_timepoints, target_power):
    """Run the power analysis with the specified parameters"""
    display_clinical_power_analysis(
        example_N=example_N,
        baseline_rate=baseline_rate,
        reduction_percent=reduction_percent,
        target_power=target_power,
        sig_level=0.05,
        attrition_rate=attrition_rate,
        attrition_timepoints=attrition_timepoints
    )

interactive(children=(IntText(value=181, description='Sample size per arm:', style=DescriptionStyle(descriptio…